# Red Neuronal Convencional (MLP)
#### Este notebook implementa una red totalmente conectada (sin convolución) usando PyTorch.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### 1. Transformaciones y carga de datos

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),                    # Convertir a tensor
    transforms.Normalize((0.1307,), (0.3081,)) # Normalizar (media y desviación de MNIST)
])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=1000, shuffle=False)


100.0%
100.0%
100.0%
100.0%


#### ¿Qué es MNIST?
##### Un dataset muy usado en Machine Learning. 
##### Contiene 70.000 imágenes de dígitos escritos a mano (0–9):
##### Tamaño: 28 x 28 píxeles, 60.000 imágenes de entrenamiento, 10.000 imágenes de prueba, Blanco y negro (1 canal)

### 3. Definir el modelo MLP

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.modelo = nn.Sequential(
            nn.Flatten(),              # nn.Flatten() convierte la imagen 2D (28x28) en un vector 1D (784)
            nn.Linear(784, 256),       # Capa oculta 1 (es una capa densa (X entradas, Y salidas)
            nn.ReLU(),                 # Activa las neuronas (introduce no linealidad)
            nn.Linear(256, 128),       # Capa oculta 2
            nn.ReLU(),
            nn.Linear(128, 10)         # Capa de salida, La última capa tiene 10 neuronas, una por cada clase (dígito del 0 al 9)
        )

    def forward(self, x):
        return self.modelo(x)

### 4. Instanciar el modelo, función de pérdida y optimizador

In [12]:
modelo = MLP()
criterio = nn.CrossEntropyLoss()
optimizador = optim.Adam(modelo.parameters(), lr=0.001)

### 5. Entrenamiento

In [14]:
# El modelo predice (forward), Se hace retropropagación con .backward() por 5 epocas.
for epoca in range(5):
    modelo.train()
    perdida_total = 0
    for datos, etiquetas in train_loader:       #Se calcula la pérdida (error) entre la predicción y la etiqueta verdadera
        optimizador.zero_grad()
        salida = modelo(datos)
        perdida = criterio(salida, etiquetas)
        perdida.backward()
        optimizador.step()                      #Se ajustan los pesos con optimizador.step()
        perdida_total += perdida.item()
    print(f"Época {epoca+1} - Pérdida promedio: {perdida_total / len(train_loader):.4f}")

Época 1 - Pérdida promedio: 0.0315
Época 2 - Pérdida promedio: 0.0294
Época 3 - Pérdida promedio: 0.0260
Época 4 - Pérdida promedio: 0.0214
Época 5 - Pérdida promedio: 0.0188


### 6. Evaluación

In [16]:
# Se mide cuántas predicciones son correctas comparando con las etiquetas verdaderas
modelo.eval()
correctos = 0
total = 0
with torch.no_grad():
    for datos, etiquetas in test_loader:
        salida = modelo(datos)
        _, pred = torch.max(salida, 1)
        correctos += (pred == etiquetas).sum().item()
        total += etiquetas.size(0)

print(f"Precisión en test: {100 * correctos / total:.2f}%")

Precisión en test: 97.79%
